In [27]:
import mlflow
from lib.reproduction import training_info

mlflow.set_tracking_uri("http://localhost:5000")

In [28]:
client = mlflow.MlflowClient()

kv = {}
for oxide, v in training_info.items():
    sub = {}
    for comp_range_name, vv in v.items():
        model_name = f"{oxide}_{comp_range_name}_outlier_removal"
        models = client.search_model_versions(f"name = '{model_name}'")
        model = mlflow.pyfunc.load_model(models[0].source)
        sub[comp_range_name] = model
        
    kv[oxide] = sub


kv

{'SiO2': {'Full': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: 0aaffb8431b04258b17347bc32f3f1e1,
  'Low': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: 67370d7540c847c49659c81f5465c5a0,
  'Mid': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: 48cd9dc1d5564885b4d74ee9ee2112b6,
  'High': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: b47d6ac592d44e939e0fb7554fb6eb81},
 'TiO2': {'Full': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: b7a3fe6e1b88432f9f947533aa0da7a8,
  'Low': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: 6b3bd56d61484cf29adbd882f9bf916d,
  'Mid': mlflow.pyfunc.loaded_model:
    artifact_path: model
    flavor: mlflow.sklearn
    run_id: 7d88f76b992347dcb9c4fdb91419b402,
  'High': mlflow.pyfunc.loaded_mode

In [29]:
import dotenv
from lib.data_handling import load_data
env = dotenv.dotenv_values(dotenv.find_dotenv())

dataset_loc = env.get('DATA_PATH') or ""
print(dataset_loc)

smol_data = load_data(
    "../data/data/calib/calib_2015/1600mm/pls/",
    10
)

data/data/calib/calib_2015/1600mm/pls


Loading data: 100%|██████████| 10/10 [00:02<00:00,  4.49it/s]


In [30]:
from lib.data_handling import CustomSpectralPipeline
from lib.reproduction import masks, major_oxides

compositon_data_path = env.get('COMPOSITION_DATA_PATH') or ""
print(compositon_data_path)

pipeline = CustomSpectralPipeline(
    masks,
    "../data/data/calib/ccam_calibration_compositions.csv",
    major_oxides
)

processed_data = pipeline.fit_transform(smol_data)
processed_data.head()

data/data/calib/ccam_calibration_compositions.csv


Transforming samples: 100%|██████████| 10/10 [00:00<00:00, 12.53it/s]


,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,848.89642,SiO2,TiO2,Al2O3,FeOT,MgO,CaO,Na2O,K2O,Sample Name
0,3.998589e+11,3.623102e+11,3.125834e+11,2.548143e+11,2.384109e+11,2.535727e+11,2.867055e+11,3.181519e+11,3.228308e+11,3.108220e+11,...,2.293541e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
1,3.101115e+11,2.760840e+11,2.294153e+11,1.789714e+11,1.602546e+11,1.737365e+11,2.033433e+11,2.307903e+11,2.380900e+11,2.286484e+11,...,2.224658e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
2,4.784395e+11,4.364859e+11,3.787261e+11,3.007351e+11,2.869929e+11,3.060502e+11,3.437366e+11,3.862550e+11,3.896103e+11,3.750934e+11,...,2.239592e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
3,5.402266e+11,4.974165e+11,4.266432e+11,3.354670e+11,3.232402e+11,3.433198e+11,3.847175e+11,4.298031e+11,4.313859e+11,4.173089e+11,...,1.634240e+09,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
4,4.925927e+11,4.457917e+11,3.862441e+11,3.063318e+11,2.987273e+11,3.191648e+11,3.567172e+11,3.909940e+11,3.978301e+11,3.836622e+11,...,2.234164e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399


In [31]:
from lib.norms import Norm1Scaler, Norm3Scaler
import lib.norms
import importlib
importlib.reload(lib.norms)

drop_cols = major_oxides + ['Sample Name']

scaler = Norm1Scaler(reshaped=True)
X = scaler.fit_transform(processed_data.drop(drop_cols, axis=1))

y = processed_data['SiO2']
X[0:5]

,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,847.10272,847.30212,847.50153,847.70081,847.90009,848.09943,848.29871,848.49799,848.6972,848.89642
0,0.000583,0.000528,0.000456,0.000371,0.000348,0.000370,0.000418,0.000464,0.000471,0.000453,...,0.000036,0.000034,0.000032,0.000031,0.000030,0.000030,3.003050e-05,3.051429e-05,0.000031,0.000033
1,0.000515,0.000458,0.000381,0.000297,0.000266,0.000288,0.000338,0.000383,0.000395,0.000380,...,0.000039,0.000037,0.000035,0.000034,0.000034,0.000034,3.333443e-05,3.391175e-05,0.000035,0.000037
2,0.000778,0.000710,0.000616,0.000489,0.000467,0.000498,0.000559,0.000628,0.000634,0.000610,...,0.000038,0.000036,0.000034,0.000034,0.000033,0.000034,3.290123e-05,3.329541e-05,0.000034,0.000036
3,0.000794,0.000731,0.000627,0.000493,0.000475,0.000505,0.000565,0.000632,0.000634,0.000613,...,0.000009,0.000006,0.000004,0.000003,0.000002,0.000001,8.131556e-07,6.993754e-07,0.000001,0.000002
4,0.000755,0.000683,0.000592,0.000469,0.000458,0.000489,0.000546,0.000599,0.000609,0.000588,...,0.000036,0.000034,0.000032,0.000032,0.000031,0.000031,3.073763e-05,3.115510e-05,0.000032,0.000034


In [32]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X[:1])
predictions
# out = mean_squared_error(predictions, y[:1], squared=False)
# out/100


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


array([1.43265354])

In [33]:
def get_weights(y_full, blend_range_min, blend_range_max):
    """
    Helper function to calculate weights for blending predictions.
    """
    w_upper = (y_full - blend_range_min) / (blend_range_max - blend_range_min)
    w_lower = 1 - w_upper
    return w_lower, w_upper


def predict_composition_with_blending(oxide, y_full, X, models, optimized_blending_ranges):
    """
    Predicts the composition of the given oxide based on the full model prediction (y_full)
    and the optimized blending ranges, including blending between "mid" and "high" models as well as "low" and "mid" models.
    """
    # Check for non-blending range predictions first
    blend_ranges = ["Low-Mid", "Mid-High"]

    for range_name, (range_min, range_max) in optimized_blending_ranges[oxide].items():
        if range_min <= y_full <= range_max and range_name not in blend_ranges:
            return models[oxide][range_name](y_full)

    for blend_range in blend_ranges:
        # Check if blend_range is defined for the given oxide
        if blend_range in optimized_blending_ranges[oxide]:
            blend_range_min, blend_range_max = optimized_blending_ranges[oxide][blend_range]

            # Check if y_full is within the defined blending range
            if blend_range_min <= y_full <= blend_range_max:
                w_lower, w_upper = get_weights(y_full, blend_range_min, blend_range_max)

                y_lower = models[oxide][blend_range.split("-")[0]].predict(X)
                y_upper = models[oxide][blend_range.split("-")[1]].predict(X)

                y_final = w_lower * y_lower + w_upper * y_upper
                print(f"y_lower: {y_lower}, y_upper: {y_upper}, y_final: {y_final}")

                return y_final

    # Error if y_full is outside any defined range
    raise ValueError(f"y_full value {y_full} for oxide {oxide} is outside the defined blending ranges.")

In [34]:
from lib.reproduction import optimized_blending_ranges  # noqa: E402

y_full = kv['SiO2']['Full'].predict(X[:1])
print(y_full)
predict_composition_with_blending('SiO2', y_full, X[:1], kv, optimized_blending_ranges)

[54.13805783]
y_lower: [3792.26896365], y_upper: [416.18535408], y_final: [548.21475175]


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


array([548.21475175])